<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SageMaker Machine Learning engine

This notebook shows how to log the payload for a model deployed on SageMaker using Watson OpenScale python sdk.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Feedback logging](#feedback)
- [6. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.0 Sample model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

### 1.1 Installation and authentication

In [ ]:
!pip install sagemaker --no-cache | tail -n 1

In [ ]:
!pip install ibm-ai-openscale==2.1.1 --no-cache | tail -n 1

### **Action:** Restart the kernel.

Import and initiate.

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.supporting_classes import PayloadRecord
from ibm_ai_openscale.supporting_classes.enums import InputDataType, ProblemType
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *

#### ACTION: Get Watson OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

How to get your Watson OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing Watson OpenScale instance
```
ibmcloud target -g <myResourceGroup>
```
- get details of the instance
```
ibmcloud resource service-instance 'Watson-OpenScale-instance_name'
```

#### Let's define some constants required to set up data mart:

- WATSON_OS_CREDENTIALS
- POSTGRES_CREDENTIALS
- SCHEMA_NAME

In [ ]:
WATSON_OS_CREDENTIALS = {
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "instance_guid": "****",
  "apikey": "****"
}

In [ ]:
POSTGRES_CREDENTIALS = {
  "connection": {
    "cli": {
      "arguments": [
        [
          "host=****.databases.appdomain.cloud port=31173 dbname=ibmclouddb user=**** sslmode=verify-full"
        ]
      ],
      "bin": "psql",
      "certificate": {
        "certificate_base64": "****",
        "name": "****"
      },
      "composed": [
        "PGPASSWORD=**** PGSSLROOTCERT=**** psql 'host=****.databases.appdomain.cloud port=31173 dbname=ibmclouddb user=ibm_cloud_*** sslmode=verify-full'"
      ],
      "environment": {
        "PGPASSWORD": "****",
        "PGSSLROOTCERT": "****"
      },
      "type": "cli"
    },
    "postgres": {
      "authentication": {
        "method": "direct",
        "password": "****",
        "username": "ibm_cloud_****"
      },
      "certificate": {
        "certificate_base64": "****",
        "name": "****"
      },
      "composed": [
        "postgres://ibm_cloud_***:***.****.databases.appdomain.cloud:31173/ibmclouddb?sslmode=verify-full"
      ],
      "database": "ibmclouddb",
      "hosts": [
        {
          "hostname": "****.databases.appdomain.cloud",
          "port": 31173,
          "protocol": "postgres"
        }
      ],
      "path": "/ibmclouddb",
      "query_options": {
        "sslmode": "verify-full"
      },
      "scheme": "postgres",
      "type": "uri"
    }
  },
  "instance_administration_api": {
    "deployment_id": "crn:v1:bluemix:public:databases-for-postgresql:us-south:a/****::",
    "instance_id": "crn:v1:bluemix:public:databases-for-postgresql:us-south:a/****::",
    "root": "https://api.******.databases.cloud.ibm.com/v4/ibm"
  }
}

In [ ]:
SCHEMA_NAME = 'data_mart_for_aws_sagemaker'

Create the schema for data mart.

In [ ]:
create_postgres_schema(postgres_credentials=POSTGRES_CREDENTIALS, schema_name=SCHEMA_NAME)

In [ ]:
client = APIClient(WATSON_OS_CREDENTIALS)

In [ ]:
client.version

### 1.2 DataMart setup

>NOTE: If you have already created a data_mart and need to delete it, uncomment and run the cell below:

In [ ]:
#client.data_mart.delete()

In [ ]:
client.data_mart.setup(db_credentials=POSTGRES_CREDENTIALS, schema=SCHEMA_NAME)

In [ ]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind  `SageMaker` machine learning engine

Provide credentials using the following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [ ]:
SAGEMAKER_ENGINE_CREDENTIALS = {'access_key_id': '****', 
                   'secret_access_key': '****', 
                   'region': '****'}

In [ ]:
binding_uid = client.data_mart.bindings.add('My SageMaker engine', SageMakerMachineLearningInstance(SAGEMAKER_ENGINE_CREDENTIALS))

In [ ]:
bindings_details = client.data_mart.bindings.get_details()

In [ ]:
client.data_mart.bindings.list()

<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add subscriptions

List available deployments.

**Note:** Depending on the number of assets, it may take some time.

In [ ]:
client.data_mart.bindings.list_assets()

**Action:** Assign your source_uid to `source_uid` variable below.

In [ ]:
source_uid = '****'

In [ ]:
subscription = client.data_mart.subscriptions.add(
    SageMakerMachineLearningAsset(
                source_uid=source_uid,
                binding_uid=binding_uid,
                input_data_type=InputDataType.STRUCTURED,
                problem_type=ProblemType.MULTICLASS_CLASSIFICATION,
                label_column='diagnosis',
                probability_column='score',
                prediction_column='predicted_label'))

#### Get subscriptions list

In [ ]:
subscriptions = client.data_mart.subscriptions.get_details()

In [ ]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

#### List subscriptions

In [ ]:
client.data_mart.subscriptions.list()

<a id="scoring"></a>
## 4. Scoring and payload logging

### 4.1 Score the product line model and measure response time

In [ ]:
import requests
import time
import json
import boto3

binding_details = client.data_mart.bindings.get_details(binding_uid=binding_uid)
subscription_details = subscription.get_details()

access_id = binding_details['entity']['credentials']['access_key_id']
access_key = binding_details['entity']['credentials']['secret_access_key']
region = binding_details['entity']['credentials']['region']
endpoint_name = subscription_details['entity']['deployments'][0]['name']
runtime = boto3.client('sagemaker-runtime', region_name=region, aws_access_key_id=access_id, aws_secret_access_key=access_key)

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
        
payload = "17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482\n17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482"

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

### 4.2 Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with Watson OpenScale standard.

In [ ]:
values = []

for v in payload.split('\n'):
    values.append([float(s) for s in v.split(',')])

request_data = {'fields': fields, 'values': values}

response_data = {'fields': list(result['predictions'][0]),
            'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [ ]:
records_list = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

for i in range(1, 10):
    records_list.append(PayloadRecord(request=request_data, response=response_data, response_time=response_time))

subscription.payload_logging.store(records=records_list)

#### Store the payload using REST API

Get the token first.

In [ ]:
token_endpoint = "https://iam.bluemix.net/identity/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

data = {
    "grant_type":"urn:ibm:params:oauth:grant-type:apikey",
    "apikey":WATSON_OS_CREDENTIALS["apikey"]
}

req = requests.post(token_endpoint, data=data, headers=headers)
token = req.json()['access_token']

Store the payload.

In [ ]:
import requests, uuid

PAYLOAD_STORING_HREF_PATTERN = '{}/v1/data_marts/{}/scoring_payloads'
endpoint = PAYLOAD_STORING_HREF_PATTERN.format(WATSON_OS_CREDENTIALS['url'], WATSON_OS_CREDENTIALS['data_mart_id'])

payload = [{
    'binding_id': binding_uid, 
    'deployment_id': subscription.get_details()['entity']['deployments'][0]['deployment_id'], 
    'subscription_id': subscription.uid, 
    'scoring_id': str(uuid.uuid4()), 
    'response': response_data,
    'request': request_data
}]


headers = {"Authorization": "Bearer " + token}
      
req_response = requests.post(endpoint, json=payload, headers = headers)

print("Request OK: " + str(req_response.ok))

<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [ ]:
subscription.quality_monitoring.enable(threshold=0.7, min_records=5)

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of the feedback table using below method.

In [ ]:
subscription.feedback_logging.print_table_schema()

The feedback records can be send to feedback table using the code below.

In [ ]:
feedback_records = []

fields = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
          'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se',
          'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se',
          'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst',
          'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst',
          'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'diagnosis']

for i in range(1, 10):
    feedback_records.append([17.02,23.98,112.8,899.3,0.1197,0.1496,0.2417,0.1203,0.2248,0.06382,0.6009,1.398,3.999,67.78,0.008268,0.03082,0.05042,0.01112,0.02102,0.003854,20.88,32.09,136.1,1344,0.1634,0.3559,0.5588,0.1847,0.353,0.08482, 1])

subscription.feedback_logging.store(feedback_data=feedback_records, fields=fields)

### Run quality monitoring on demand

By default, quality monitoring is run on an hourly schedule. You can also trigger it on demand using the code below.

In [ ]:
run_details = subscription.quality_monitoring.run()

Since the monitoring runs in the background you can use the method below to check the status of the job.

In [ ]:
status = run_details['status']
id = run_details['id']

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

### Show the quality metrics

In [ ]:
subscription.quality_monitoring.show_table()

Get all calculated metrics.

In [ ]:
deployment_uids = subscription.get_deployment_uids()

In [ ]:
subscription.quality_monitoring.get_metrics(deployment_uid=deployment_uids[0])

<a id="datamart"></a>
## 6. Get the logged data

### 6.1 Payload logging

#### Print schema of payload_logging table

In [ ]:
subscription.payload_logging.print_table_schema()

#### Show (preview) the table

In [ ]:
subscription.payload_logging.describe_table()

#### Return the table content as pandas dataframe

In [ ]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')

### 6.2 Feddback logging

Check the schema of the table.

In [ ]:
subscription.feedback_logging.print_table_schema()

Preview the table content.

In [ ]:
subscription.feedback_logging.show_table()

Describe the table (calulcate basic statistics).

In [ ]:
subscription.feedback_logging.describe_table()

Get the table content.

In [ ]:
feedback_pd = subscription.feedback_logging.get_table_content(format='pandas')

### 6.3 Quality metrics table

In [ ]:
subscription.quality_monitoring.print_table_schema()

In [ ]:
subscription.quality_monitoring.show_table()

### 6.4 Performance metrics table

In [ ]:
subscription.performance_monitoring.print_table_schema()

In [ ]:
subscription.performance_monitoring.show_table()

### 6.5 Data Mart measurement facts table

In [ ]:
client.data_mart.get_deployment_metrics()

---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.